In [ ]:
import pandas as pd
import pylab as pl
import seaborn as sn
import plotly.express as px
pl.style.use('bmh')
pd.options.mode.chained_assignment = None

T=pd.read_csv(r'C:\Users\julia\Documents\python36\datasource\data\arbres_paris.csv', sep=';', encoding='latin1')
T.head()

In [ ]:
fig = px.density_mapbox(
data_frame=T,
lat='LAT', lon='LON',
radius=1,
color_continuous_scale=[(0, 'green'), (0.5, 'red'), (1, 'black')],
opacity=0.8,
hover_name='NOM'
)

fig.update_layout(
mapbox={
'style': 'stamen-terrain',
'center': {'lat': T['LAT'].mean(), 'lon': T['LON'].mean()},
'zoom': 11
},
showlegend=False,
coloraxis_showscale=False
)

fig.show()

In [ ]:
L = ['Frêne', 'Olivier', 'Bouleau', 'Noisetier de Byzance', 'Charme', 'Cyprès', 'Marier', 'Platane', 'Aulne']

In [ ]:
T.query('DOMANIALITE == "Alignement"')['NOM'].value_counts()

In [ ]:
def allergisant(nom):
    if nom in L:
        return 1
    else:
        return 0

In [ ]:
T['Allergisant'] = T['NOM'].apply(allergisant)
T.head()

In [ ]:
T['Allergisant'] .mean()

In [ ]:
T.query('DOMANIALITE == "Alignement"').mean()

# Tree Density

In [ ]:
import geopandas as gp

J=gp.read_file(r'C:\Users\julia\Documents\python36\datasource\data\Arr_paris.geojson')
J=J.set_index('arrondissement')
J.head()

In [ ]:
N=T.groupby('ARRONDISSEMENT')['NOM'].count()
N

In [ ]:
J['Densité']=N / J['superficie_km2']
J.head()

In [ ]:
J=J.reset_index()
J.head()

In [ ]:
fig1 = px.choropleth(
    data_frame=J,
    locations='arrondissement',
    geojson=J,
    featureidkey='properties.arrondissement',
    color='Densité',
    color_continuous_scale=[(0, 'black'), (0.5, 'orange'), (1, 'green')]
)


fig1.update_layout(
    height=400,
    width=900,
    coloraxis={'colorbar': {'len': 0.6, 'x': 0.8, 'y': 0.7, 'title': '<b>Arbres / km2 </b>'}},
    title={'text': "<b>Densité des arbres Paris</bsn>", 'font': {'family': 'comic', 'color': 'black', 'size': 15}, 'x': 0.95, 'y': 0.1},
    showlegend=False,
    margin={'r': 0, 't': 0, 'l': 0, 'b': 0}
)

fig1.update_geos(fitbounds='locations', projection={'type': 'transverse mercator'})

fig1.add_scattergeo(
    lat=J['geometry'].centroid.y,
    lon=J['geometry'].centroid.x - 0.005,
    mode='markers+text',
    marker={'symbol': 'square', 'size': 12, 'opacity': 1, 'color': 'linen'},
    text=J['arrondissement'].str.replace('PARIS', ''),
    textfont={'color': 'black', 'family': 'Arial', 'size': 14},
    textposition='middle center',
    showlegend=False
)


fig1.show()

# Allergic tree

In [ ]:
J=J.set_index('arrondissement')
J.head()

In [ ]:
A=T.groupby('ARRONDISSEMENT')['Allergisant'].sum()

In [ ]:
J['Proportion']=A/N
J.head()

In [ ]:
J=J.reset_index()

In [ ]:
J['%_A1']=[str(x*100)[:2] + '%' for x in J['Proportion']]
J.head()

# Density / Proportion

In [ ]:
S=J.query('Proportion>0.273')

In [ ]:
S.head()

In [ ]:

fig1.add_scattergeo(
    lat=S['geometry'].centroid.y - 0.002,
    lon=S['geometry'].centroid.x + 0.005,
    mode='markers+text',
    marker={'size': 1},
    text=S['%_A1'],
    textfont={'color': 'red', 'size': 14, 'family': 'Arial Black'},
    textposition='middle center'
)

# update the layout to add a title
fig1.update_layout(
    title={'text': "<b>Pourcentages d'arbres allergisants <br> par arrondissement</b>"}
)

In [ ]:
L=J.query('Proportion<=0.273')

In [ ]:
L.head()

In [ ]:
fig1.add_scattergeo(
    lat=L['geometry'].centroid.y - 0.002,
    lon=L['geometry'].centroid.x + 0.005,
    mode='markers+text',
    marker={'size': 1},
    text=L['%_A1'],
    textfont={'color': 'red', 'size': 14, 'family': 'Arial Black'},
    textposition='middle center'
)

fig1.update_layout(
    title={'text': "<b>Pourcentages d'arbres allergisants <br> par arrondissement </b>", 'font': {'family': 'comic', 'color': 'black', 'size': 15}, 'x': 0.95, 'y': 0.05}
)

fig1.show()

In [ ]:
fig2 = px.choropleth(
    data_frame=J,
    locations='arrondissement',
    geojson=J,
    featureidkey='properties.arrondissement',
    color='Densité',
    color_continuous_scale=[(0, 'black'), (0.5, 'orange'), (1, 'green')]
)


fig2.update_layout(
    height=400,
    width=900,
    coloraxis={'colorbar': {'len': 0.6, 'x': 0.8, 'y': 0.7, 'title': '<b>Arbres / km2 </b>'}},
    title={'text': "<b>Densité des arbres Paris</bsn>", 'font': {'family': 'comic', 'color': 'black', 'size': 15}, 'x': 0.95, 'y': 0.1},
    showlegend=False,
    margin={'r': 0, 't': 0, 'l': 0, 'b': 0}
)

fig2.update_geos(fitbounds='locations', projection={'type': 'transverse mercator'})

fig2.add_scattergeo(
    lat=J['geometry'].centroid.y,
    lon=J['geometry'].centroid.x - 0.005,
    mode='markers+text',
    marker={'symbol': 'square', 'size': 12, 'opacity': 1, 'color': 'linen'},
    text=J['arrondissement'].str.replace('PARIS', ''),
    textfont={'color': 'black', 'family': 'Arial', 'size': 14},
    textposition='middle center',
    showlegend=False
)



fig2.add_scattergeo(
    lat=L['geometry'].centroid.y - 0.002,
    lon=L['geometry'].centroid.x + 0.005,
    mode='markers+text',
    marker={'size': 1},
    text=L['%_A1'],
    textfont={'color': 'red', 'size': 14, 'family': 'Arial Black'},
    textposition='middle center'
)

# update the layout to add a title
fig2.update_layout(
    title={'text': "<b>Pourcentages d'arbres allergisants <br> par arrondissement</b>"}
)